In [1]:
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import os
import json
import test
import create_result
import create_reference
import pickle
from configuration import DataConfig

data_config = DataConfig().config

create_reference.create_reference()

all_models = list(set([i.split(".")[0] for i in os.listdir(data_config["checkpoint_dir"]) if "model" in i]))
all_models.sort(key=lambda x: int(x[6:]))

all_model_paths = [os.path.join(data_config["checkpoint_dir"],i) for i in all_models]
results = []


In [14]:
with open("./Results/results.pkl","rb") as fl:
    result_batch = pickle.load(fl)

In [15]:
results = result_batch

In [16]:
batch = 0

In [18]:
for batch in range(5,len(all_model_paths),5):
    result_batch = test.main(all_model_paths[batch:(min(batch+5,len(all_model_paths)))],"test",64,)
    results += result_batch
    with open(os.path.join(data_config["result_dir"],"results.pkl"),"wb") as fl:
        pickle.dump(results,fl)
    print("Saving Results for :")
    print(all_models[batch:(min(batch+5,len(all_model_paths)))])
    for model,result in zip(all_models[batch:(min(batch+5,len(all_model_paths)))],result_batch):
        with open(os.path.join(data_config["result_dir"],model+"_result.json"),"w") as fl:
            fl.write(json.dumps(result, indent=4, sort_keys=True))
        create_result.create(model+"_generated.json",model+"_result.json",data_config)

Restoring weights from ./models/train/model-3036
INFO:tensorflow:Restoring parameters from ./models/train/model-3036
Processing batch 1
Processing batch 2
Processing batch 3
Processing batch 4
Processing batch 5
Processing batch 6
Processing batch 7
Processing batch 8
Processing batch 9
Processing batch 10
Processing batch 11
Processing batch 12
Processing batch 13
Processing batch 14
Processing batch 15
Processing batch 16
Processing batch 17
Processing batch 18
Processing batch 19
Processing batch 20
Processing batch 21
Processing batch 22
Processing batch 23
Processing batch 24
Processing batch 25
Processing batch 26
Processing batch 27
Processing batch 28
Processing batch 29
Processing batch 30
Processing batch 31
Processing batch 32
Processing batch 33
Processing batch 34
Processing batch 35
Processing batch 36
Processing batch 37
Processing batch 38
Processing batch 39
Processing batch 40
Processing batch 41
Processing batch 42
Processing batch 43
Processing batch 44
Processing b

KeyboardInterrupt: 

In [19]:
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import os
import tensorflow as tf
import numpy as np
import time
from tensorflow.python.client import device_lib

from model import Model_S2VT
from data_generator import Data_Generator
from inference_util import Inference

import inception_base
import configuration
import argparse


In [20]:
data_config = configuration.DataConfig().config
data_gen = Data_Generator(processed_video_dir = data_config["processed_video_dir"],
                         caption_file = data_config["caption_file"],
                         unique_freq_cutoff = data_config["unique_frequency_cutoff"],
                         max_caption_len = data_config["max_caption_length"])

data_gen.load_vocabulary(data_config["caption_data_dir"])
data_gen.load_dataset(data_config["caption_data_dir"])
#data_gen.build_dataset()

model_config = configuration.ModelConfig(data_gen).config
model = Model_S2VT( num_frames = model_config["num_frames"],
                    image_width = model_config["image_width"],
                    image_height = model_config["image_height"],
                    image_channels = model_config["image_channels"],
                    num_caption_unroll = model_config["num_caption_unroll"],
                    num_last_layer_units = model_config["num_last_layer_units"],
                    image_embedding_size = model_config["image_embedding_size"],
                    word_embedding_size = model_config["word_embedding_size"],
                    hidden_size_lstm1 = model_config["hidden_size_lstm1"],
                    hidden_size_lstm2 = model_config["hidden_size_lstm2"],
                    vocab_size = model_config["vocab_size"],
                    initializer_scale = model_config["initializer_scale"],
                    learning_rate = model_config["learning_rate"],
                    mode="train",
                    rnn1_input_keep_prob=model_config["rnn1_input_keep_prob"],
                    rnn1_output_keep_prob=model_config["rnn1_output_keep_prob"],
                    rnn2_input_keep_prob=model_config["rnn2_input_keep_prob"],
                    rnn2_output_keep_prob=model_config["rnn2_output_keep_prob"]
                    )
model.build()

In [21]:
tf.trainable_variables()

[<tf.Variable 'image_encoding/weights:0' shape=(1536, 2000) dtype=float32_ref>,
 <tf.Variable 'image_encoding/biases:0' shape=(2000,) dtype=float32_ref>,
 <tf.Variable 'image_encoding/beta:0' shape=(2000,) dtype=float32_ref>,
 <tf.Variable 'image_encoding/gamma:0' shape=(2000,) dtype=float32_ref>,
 <tf.Variable 'embedding/word_embedding:0' shape=(12974, 1000) dtype=float32_ref>,
 <tf.Variable 'RNN_1/rnn/basic_lstm_cell/weights:0' shape=(3000, 4000) dtype=float32_ref>,
 <tf.Variable 'RNN_1/rnn/basic_lstm_cell/biases:0' shape=(4000,) dtype=float32_ref>,
 <tf.Variable 'RNN_2/rnn/basic_lstm_cell/weights:0' shape=(3000, 4000) dtype=float32_ref>,
 <tf.Variable 'RNN_2/rnn/basic_lstm_cell/biases:0' shape=(4000,) dtype=float32_ref>,
 <tf.Variable 'word_decoding/weights:0' shape=(1000, 12974) dtype=float32_ref>,
 <tf.Variable 'word_decoding/biases:0' shape=(12974,) dtype=float32_ref>]

In [22]:
ans = (12974*1000 + 2*(3000*4000 + 4000) + 1000*12974 + 12974)

In [24]:
(((ans * 32)/8)/1024)/1024

190.61650848388672